# **Precio promedio - Recámaras**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre and "2024" in nombre

    #if nombre.startswith("df_") and "pachuca" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Preciol_23':'precio','Baño_total': 'baño_total','Banos_Total':'baño_total','Precio':'precio','recamaras':'Recamaras'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'Categoria', 'precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'baño_total', 'Seguridad',
       'Colonia', 'CP', 'url']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
# Concatenar todos los DataFrames individuales en uno solo
df_concatenado = pd.concat(dataframes_list)

## Eliminación de duplicados

In [4]:
# Eliminar por duplicado general, osea los registros exactamente iguales
df_concatenado.drop_duplicates(inplace=True)

In [5]:
# Agrupar por los registros duplicados y contar las ocurrencias
agrupados = df_concatenado[df_concatenado.duplicated(subset=['metros_construido','Colonia', 'CP', 'precio','Estacionamientos','Recamaras','propiedad'], keep=False)].groupby(['metros_construido','Colonia', 'CP', 'precio','Recamaras','propiedad']).size().reset_index(name='count')
print(agrupados)

     metros_construido                  Colonia       CP   precio  Recamaras  \
0                   50             San Fernando  42186.0   650000          2   
1                   54     Lopez Mateos 1a Secc    42094   630000          3   
2                   70        bosques del penar    42034   950000          2   
3                   74  Jardines de la Hacienda    42088  1480000          2   
4                   74              San Antonio  42083.0  1100000          3   
..                 ...                      ...      ...      ...        ...   
124                380                cabanitas    42034   750000          4   
125                380                cabanitas    42034   750000          4   
126                410              El Tezontle  42084.0  7900000          4   
127                416               Periodista  42060.0  8500000          4   
128               1300     San Pedro Nopalcalco  42084.0   680000          7   

                                       

In [6]:
# Eliminar duplicados, conservando la primera aparición
df_sin_duplicados = df_concatenado.drop_duplicates(subset=['metros_construido','Colonia', 'CP', 'precio','Estacionamientos','Recamaras','baño_total','propiedad'], keep='first')
# conservar la última aparición
# df_sin_duplicados = df_concatenado.drop_duplicates(keep='last')
print(df_sin_duplicados)

        id Categoria   precio  \
0        1        S3  8500000   
1        2        S1  4500000   
2        3        S3  8600000   
3        4        S3  8600000   
4        5        E2   615000   
...    ...       ...      ...   
1801  1802        E2   607500   
1802  1803        E3   768131   
1803  1804        E3   768131   
1804  1805        E3   800708   
1805  1806        E3   800708   

                                              propiedad  metros_total  \
0     Amplia Casa en Venta en Col. Periodistas-Pachu...           480   
1     VENTA DE CASA, ANT CARR PACHUCA R. DEL MONTE C...           410   
2     Casa en venta en Zona Plateada, Real del Monte...           256   
3     Hermosa casa reaidencial en zona exclusiva de ...           256   
4     Casa en Col. Puerta de Hierro, Hgo., Remate!!!...           180   
...                                                 ...           ...   
1801   casa duplex en venta fraccionamiento el saucillo            57   
1802  descubre la e

## Diidir por tipo

In [7]:
df_sin_duplicados['Tipo'].unique()

array(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio',
       'Departamento', 'Penthouse', 'Estudio', 'Condominio Horizontal',
       'Duplex', 'Casa ', 'Daoplex'], dtype=object)

In [8]:
df_sin_duplicados['Tipo'] = df_sin_duplicados['Tipo'].replace('Daoplex', 'Duplex')

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17472\733932996.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sin_duplicados['Tipo'] = df_sin_duplicados['Tipo'].replace('Daoplex', 'Duplex')


In [9]:
casas = df_sin_duplicados[df_sin_duplicados['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio', 'Casa '])]
depas = df_sin_duplicados[df_sin_duplicados['Tipo'].isin(['Departamento', 'Condominio Horizontal','Duplex', 'Penthouse', 'Estudio'])]

# Resultados

## Casas

In [10]:
casas['Recamaras'] = casas['Recamaras'].astype(int)
casas = casas[(casas['Recamaras'] >= 1) & (casas['Recamaras'] <= 8)]
casas['Recamaras'].unique()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17472\1015311021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  casas['Recamaras'] = casas['Recamaras'].astype(int)


array([3, 4, 5, 2, 8, 6, 7, 1])

In [11]:
# Ajustar los valores para que no superen un #
casas.loc[:, 'Recamaras'] = casas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_casas = casas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_casas

,Recamaras,precio
0,1,1.071034e+06
1,2,1.394700e+06
2,3,2.907776e+06
3,4,4.077552e+06
4,5,5.866806e+06
5,6,5.813704e+06
6,7,2.819038e+06
7,8,3.313056e+06


In [12]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_casas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_casas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [18]:
casas = casas.drop_duplicates(subset=['precio', 'Recamaras', 'Tipo'])
casas[['precio','Recamaras']].sort_values(by='precio')

,precio,Recamaras
528,0,3
162,0,3
1641,4800,4
1602,55000,4
0,291684,3
...,...,...
1038,26000000,5
1348,27000000,4
1,28500000,4
0,32000000,6


In [19]:
# Ajustar los valores para que no superen un #
casas.loc[:, 'Recamaras'] = casas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_casas = casas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_casas

,Recamaras,precio
0,1,1.162626e+06
1,2,1.612339e+06
2,3,3.062640e+06
3,4,4.608691e+06
4,5,6.345863e+06
5,6,5.797930e+06
6,7,3.260625e+06
7,8,3.205000e+06


In [20]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_casas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_casas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [21]:
# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    import os
    import plotly.io as pio

    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Ejemplo de uso
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_recamaras_casa',  carpeta='assets/graficas')


## Depa

In [13]:
depas['Recamaras'] = depas['Recamaras'].astype(int)
depas = depas[(depas['Recamaras'] >= 1) & (depas['Recamaras'] <= 8)]
depas['Recamaras'].unique()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17472\1542590914.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([6, 3, 2, 1, 4])

In [14]:
# Ajustar los valores para que no superen un #
depas.loc[:, 'Recamaras'] = depas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_depas = depas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_depas

,Recamaras,precio
0,1,2.614515e+06
1,2,1.977814e+06
2,3,3.861965e+06
3,4,2.009120e+06
4,6,3.600000e+06


In [24]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_depas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_depas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [25]:
depas = depas.drop_duplicates(subset=['precio', 'Recamaras', 'Tipo'])
depas[['precio','Recamaras']].sort_values(by='precio')

,precio,Recamaras
1623,320000,2
111,346040,2
630,420000,2
1613,444000,2
1378,486540,2
...,...,...
560,7500000,3
45,12500000,3
1310,14000000,3
35,15000000,3


In [33]:
registro_especifico = depas[depas['precio']==15000000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
259,15000000,3
35,15000000,3


In [34]:
#depas = depas[~((depas['precio']==9000000) & (depas['Recamaras']==1))]
#depas = depas[~((depas['precio']==6400000) & (depas['Recamaras']==1))]
#depas = depas[~((depas['precio']==6115945) & (depas['Recamaras']==1))]


#recam_five = depas[depas['Recamaras']==1].sort_values(by='precio')
#recam_five[['precio', 'Recamaras', 'Tipo']]

In [35]:
# Ajustar los valores para que no superen un #
depas.loc[:, 'Recamaras'] = depas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_depas = depas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_depas

,Recamaras,precio
0,1,2.701284e+06
1,2,2.228157e+06
2,3,3.825658e+06
3,4,2.271200e+06
4,6,3.850000e+06


In [36]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_depas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_depas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [37]:
# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    import os
    import plotly.io as pio

    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Ejemplo de uso
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_recamaras_depa',  carpeta='assets/graficas')
